In [64]:
from qutip import tensor, Qobj


def kron(*args: Qobj) -> Qobj:
    '''
    Kronecker product of Qobj instances.

    Parameters
    ----------

    args : Qobj
        An ordered sequence of qutip.Qobj instances.
    '''
    if len(args) == 1:
        return args[0]
    return tensor(args[0], kron(*args[1:]))

In [65]:
import warnings
from typing import Literal
import numpy as np
from scipy.linalg import ishermitian
from qutip import (Qobj, sigmaz, identity, destroy, basis, zero_ket, qzero)


class ExcitonicSystem:

    @property
    def e_el(self):
        return self._e_el

    @property
    def system_size(self):
        return self._system_size

    @property
    def dipole_moments(self):
        return self._dipole_moments

    @property
    def coupl_el(self):
        return self._coupl_el

    @property
    def state(self):
        return self._state

    @e_el.setter
    def e_el(self,
             energies: list[float] | float | np.ndarray,
             ):
        if self._state_type is None:
            if np.isscalar(energies):
                energies = np.array([energies])
            elif isinstance(energies, list):
                energies = np.array(energies)
            if not isinstance(energies, np.ndarray):
                raise ValueError(
                    f'energies are in a wrong format {type(energies)}')
            if len(energies.shape) > 1:
                raise ValueError(
                    f'energies are in a wrong shape {energies.shape}')
            self._e_el = energies
            self._system_size = energies.size
            self.__update_validity()
        else:
            raise ValueError(
                f'{type(self)} parameters cannot be changed after a state has been set.')

    @system_size.setter
    def system_size(self,
                    size: int,
                    ):
        if self._state_type is None:
            self._system_size = size
            self.__update_validity()
        else:
            raise ValueError(
                f'{type(self)} parameters cannot be changed after a state has been set.')

    @dipole_moments.setter
    def dipole_moments(self,
                       dipole_moments: list[float] | float | np.ndarray | None,
                       ):
        if self._state_type is None:
            if dipole_moments is None:
                warnings.warn(
                    'Equal dipole moments are assumed for all chromophores')
                self._dipole_moments = np.ones(self.system_size)
            else:
                if np.isscalar(dipole_moments):
                    dipole_moments = np.array([dipole_moments])
                elif isinstance(dipole_moments, list):
                    dipole_moments = np.array(dipole_moments)
                if not isinstance(dipole_moments, np.ndarray):
                    raise ValueError(
                        f'dipole_moments are in a wrong format {type(dipole_moments)}')
                if len(dipole_moments.shape) > 1:
                    raise ValueError(
                        f'dipole_moments are in a wrong shape {dipole_moments.shape}')
                self._dipole_moments = dipole_moments
            self.__update_validity()
        else:
            raise ValueError(
                f'{type(self)} parameters cannot be changed after a state has been set.')

    @coupl_el.setter
    def coupl_el(self,
                 couplings: list[list[float]] | float | np.ndarray | None,
                 ):
        if self._state_type is None:
            if couplings is None:
                warnings.warn(
                    'No coupling between chromophores is assumed')
                self._coupl_el = np.zeros((self.system_size, self.system_size))
            else:
                if np.isscalar(couplings) and self.system_size == 2:
                    couplings = np.array([0, couplings], [couplings, 0])
                elif isinstance(couplings, list) and isinstance(couplings[0], list):
                    couplings = np.array(couplings)
                if not isinstance(couplings, np.ndarray):
                    raise ValueError(
                        f'couplings is in a wrong format {type(couplings)}')
                couplings -= np.diag(np.diag(couplings))
                if ishermitian(couplings) is False:
                    raise ValueError(
                        'Couplings are not hermitian, but they should be.')
                self._coupl_el = couplings
            self.__update_validity()
        else:
            raise ValueError(
                f'{type(self)} parameters cannot be changed after a state has been set.')

    def __init__(self,
                 energies: list[float] | float | np.ndarray = None,
                 couplings: list[list[float]] | float | np.ndarray = None,
                 dipole_moments: list[float] | float | np.ndarray = None,
                 ):
        """ Create an `ExcitonicSystem` object.
        An `ExcitonicSystem` object contains the information about the excitonic part of a chromophore system intended as a collection of two-level electronic systems with a groung |0> and an excited |1> states.
        The exciton Hamiltonian is

        .. math:: H^{e} = - \sum_{i=1}^{N} \epsilon_{i}/2 \sigma^{z}_{i} + \sum_{i=1}^{N-1} \sum_{j>1}^{N} ( J_{ij} \sigma^{+}_{i} \sigma^{-}_{j} + h.c. )

        where :math:`\epsilon` are the energies and :math:`J` are the couplings.
        :math:`\sigma^{z}` is the Pauli-z operator (defined as |0><0| - |1><1|), while :math:`\sigma^{+}` (:math:`\sigma^{-}`) are the rising (lowering) ladder operators.

        Parameters
        ----------
        energies: list[float] | float | numpy.ndarray
            The electronic energy gaps of the chromphores of the network. It is a list, a number (for a single chromophore) or a monodimensional array.

        couplings: list[list[float]] | float | numpy.ndarray
            The electronic couplings between different chromophores of the network. It is in form of an Hermitian matrix with zeros in the diagonal. An off-diagonal element (i,j) represents the coupling that chromophore i experiences due to chromophore j. If the system is composed of only 2 chromophores, a float value is enough.

        dipole_moments: list[float] | float | numpy.ndarray
            The amplitude of the transition dipole moments of the chromophores. It is a list, a number (for a single chromophore) or a monodimensional array.
        """
        self._system_size = 0
        self._e_el = None
        self._dipole_moments = None
        self._coupl_el = None
        self._validity = False
        self._state_type = None
        self._state = None
        if energies is not None:
            self.electronics(energies=energies,
                             couplings=couplings,
                             dipole_moments=dipole_moments,
                             )

    def __update_validity(self):
        try:
            if self.e_el.size == self.system_size and \
                    self.dipole_moments.size == self.system_size and \
                    self.coupl_el.shape[0] == self.system_size:
                self._validity = True
            else:
                if self.e_el.size != self.system_size:
                    warnings.warn(
                        'The number of elements in energies does not match the system size.')
                if self.dipole_moments.size != self.system_size:
                    warnings.warn(
                        'The number of elements in dipole moments does not match the system size.')
                if self.coupl_el.shape[0] != self.system_size:
                    warnings.warn(
                        'The number of elements in couplings does not match the system size.')
                self._validity = False
        except:
            self._validity = False

    def electronics(self,
                    energies: list[float] | float | np.ndarray,
                    couplings: list[list[float]] | float | np.ndarray = None,
                    dipole_moments: list[float] | float | np.ndarray = None,
                    ):
        self.e_el = energies
        self.system_size = self.e_el.size
        self.dipole_moments = dipole_moments
        if self.dipole_moments.size != self.system_size:
            raise ValueError(
                'The number of elements in dipole moments does not match the system size.')
        self.coupl_el = couplings
        if self.coupl_el.shape[0] != self.system_size:
            raise ValueError(
                'The number of elements in couplings does not match the system size.')

    def set_state(self,
                  state_type: Literal['state', 'delocalized excitation',
                                      'localized excitation', 'ground'] = 'ground',
                  state: list | np.ndarray | int = 0,
                  ):
        """Set the `ExcitonicSystem` electronic ket state.
        Note that in this version of the code only pure states are accepted.

        Parameters
        ----------
        state_type : str
            Type of the input state. It must be one of the following: 'state', 'delocalized excitation', 'localized excitation', 'ground'.

        state: list | np.ndarray | int
            The state of the system. The input is given by the selected state_type:
            - `'state'`: It requires a list of coefficients for the :math:`2^{N}` states that compose the electronic Hilbert space. A standard binary ordering and local basis set are intended, that is to say, for N=3 state `[0., 0., 0., 1.+0.j, 0., 0., 0., 0.] = |011>` denotes a double excitation state with both chromophore 0 and 1 excited. 
            - `'delocalized excitation'`: It takes a list of N (complex) coefficients that describe the superposition of single excitation states. For example, for N=3, `[c_001, c_010, c_100] = [0., 1./sqrt(2), 1./sqrt(2)] = 1/sqrt(2) |010> + 1/sqrt(2) |100>`.
            - `'localized excitation'`: It takes an integer that denotes the excited chromophore. Chromophores are numbered from 0 to N-1.
            - `'ground'`: It does not require any input. The system is in the global ground state.
        """
        valid_state_types = ('state', 'delocalized excitation',
                             'localized excitation', 'ground')
        if not self._validity:
            raise ValueError(
                'Make sure system electronic parameters are specified before setting the state.')
        if state_type not in valid_state_types:
            raise ValueError(
                f'state_type must be one of the following: {valid_state_types}')
        if state_type in ('state', 'delocalized excitation'):
            if not isinstance(state, list) and isinstance(state, np.ndarray):
                raise ValueError(
                    'The state must be a list or a numpy array.')
            state = np.array(state, dtype=complex)
            if len(state.shape) > 1:
                raise ValueError(
                    'The state must be a monodimensional array.')
            expected_size = 2**self.system_size if state_type == 'state' else self.system_size
            if state.size != expected_size:
                raise ValueError(
                    'The state size must match the system size.')
            norm = np.sum(np.abs(state)**2)
            if not np.isclose(norm, 1.):
                state = state / np.sqrt(norm)
                warnings.warn(
                    'Coefficients have been normalized.')
            state = state.tolist()
        elif state_type == 'localized excitation':
            if not isinstance(state, int):
                raise ValueError('The state must be an integer.')
            if state < 0 or state >= self.system_size:
                raise ValueError(
                    'The state must be a valid chromophore index.')
        elif state_type == 'ground':
            state = 0
        self._state_type = state_type
        self._state = state

    def todict(self) -> dict:
        init_dict = self.__dict__
        sys_dict = {'class': type(self)}
        for key, value in init_dict.items():
            new_key = key[1:] if key.startswith('_') else key
            sys_dict[new_key] = value.tolist() if isinstance(
                value, np.ndarray) else value
        return sys_dict

    def get_e_Hamiltonian(self):
        '''Returns the Frenkel-exciton Hamiltonian.
        '''
        if not self._validity:
            raise ValueError(
                'The system is not valid.')
        sz = sigmaz()
        sm = destroy(2)
        sp = sm.dag()
        I = identity(2)
        H = qzero(dimensions=[2]*self.system_size)
        for i in range(self.system_size):
            H += -0.5 * self.e_el[i] * \
                kron(*[I]*(self.system_size-i-1), sz, *[I]*i)
            for j in range(i+1, self.system_size):
                H += self.coupl_el[i, j] * \
                    (kron(*[I]*(self.system_size-j-1), sm, *[I]*(j-i-1), sp, *[I]*i) +
                     kron(*[I]*(self.system_size-j-1), sp, *[I]*(j-i-1), sm, *[I]*i))
        return H

    def get_e_state(self):
        '''Returns the ket state of the system.
        '''
        if not self._validity:
            raise ValueError(
                'The system is not valid.')
        if self._state_type == 'ground':
            state = basis([2] * self.system_size, [0] * self.system_size)
        elif self._state_type == 'state':
            state = Qobj(np.array(self.state), dims=[
                         [2] * self.system_size, [1] * self.system_size], type='ket')
        elif self._state_type == 'delocalized excitation':
            state = zero_ket(dimensions=[2] * self.system_size)
            for nc, c in enumerate(self.state):
                position = list(
                    map(int, bin(1 << nc)[2:].zfill(self.system_size)))
                state += c * basis([2] * self.system_size, position)
        elif self._state_type == 'localized excitation':
            position = list(
                map(int, bin(1 << self.state)[2:].zfill(self.system_size)))
            state = basis([2] * self.system_size, position)
        return state

In [66]:
from scipy.linalg import toeplitz

epsilon = [1, 2, 3, 4]
J = toeplitz([0., 1., 0., 1.])
print(J)

esys = ExcitonicSystem(energies=epsilon, couplings=J)

[[0. 1. 0. 1.]
 [1. 0. 1. 0.]
 [0. 1. 0. 1.]
 [1. 0. 1. 0.]]


/tmp/ipykernel_11585/1454768478.py:69: UserWarning: Equal dipole moments are assumed for all chromophores
  warnings.warn(


In [67]:
print(esys.e_el)
print(esys.coupl_el)
print(esys.dipole_moments)
print(esys.system_size)

[1 2 3 4]
[[0. 1. 0. 1.]
 [1. 0. 1. 0.]
 [0. 1. 0. 1.]
 [1. 0. 1. 0.]]
[1. 1. 1. 1.]
4


In [68]:
esys.get_e_Hamiltonian()

Quantum object: dims=[[2, 2, 2, 2], [2, 2, 2, 2]], shape=(16, 16), type='oper', dtype=CSR, isherm=True
Qobj data =
[[-5.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -4.  1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1. -3.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -2.  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0. -2.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0. -1.  1.  0.  0.  1.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  1.  0.  0.  1.  0.]
 [ 0.  1.  0.  0.  1.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  1.  0.  0.  1.  1.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  2.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.  0.  2.  0.  0.  0.]
 [ 0.  0.  0. 

In [69]:
esys.set_state(state_type='delocalized excitation',
               state=[1/np.sqrt(2), 1/np.sqrt(2), 0, 0])

In [70]:
esys.get_e_state()

Quantum object: dims=[[2, 2, 2, 2], [1, 1, 1, 1]], shape=(16, 1), type='ket', dtype=Dense
Qobj data =
[[0.        ]
 [0.70710678]
 [0.70710678]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]]

In [71]:
esys.todict()

{'class': __main__.ExcitonicSystem,
 'system_size': 4,
 'e_el': [1, 2, 3, 4],
 'dipole_moments': [1.0, 1.0, 1.0, 1.0],
 'coupl_el': [[0.0, 1.0, 0.0, 1.0],
  [1.0, 0.0, 1.0, 0.0],
  [0.0, 1.0, 0.0, 1.0],
  [1.0, 0.0, 1.0, 0.0]],
 'validity': True,
 'state_type': 'delocalized excitation',
 'state': [(0.7071067811865475+0j), (0.7071067811865475+0j), 0j, 0j]}

In [ ]:
dt = 0.01
time = np.arange(0, 20, dt*20)
shots = 16000